In [1]:
push!(LOAD_PATH, "../src")
using Quaternions
using FixedSizeArrays
using RigidBodyDynamics

In [2]:
mechanism = Mechanism{Float64}("world");
world = root(mechanism)

RigidBodyDynamics.RigidBody{Float64}(RigidBodyDynamics.CartesianFrame3D("world"),#undef)

In [3]:
body1 = RigidBody(rand(SpatialInertia{Float64}, CartesianFrame3D("body1")));
joint1 = Joint("1", QuaternionFloating());
joint1ToWorld = rand(Transform3D{Float64}, joint1.frameBefore, world.frame)
body1ToJoint1 = rand(Transform3D{Float64}, body1.frame, joint1.frameAfter)
attach!(mechanism, world, joint1, joint1ToWorld, body1, body1ToJoint1);

In [4]:
joint2 = Joint("2", Revolute(rand(Vec{3, Float64})));
joint2ToBody1 = rand(Transform3D{Float64}, joint2.frameBefore, body1.frame)
body2frame = CartesianFrame3D("b2")
body2 = RigidBody(rand(SpatialInertia{Float64}, body2frame));
body2ToJoint2 = rand(Transform3D{Float64}, body2frame, joint2.frameAfter)
attach!(mechanism, body1, joint2, joint2ToBody1, body2, body2ToJoint2);

In [5]:
x = MechanismState{Float64}(mechanism);

In [6]:
rand!(x)
x.v

DataStructures.OrderedDict{RigidBodyDynamics.Joint,Array{Float64,1}} with 2 entries:
  RigidBodyDynamics.Joint… => [0.7370241129415085,0.2921916424370532,0.44886238…
  RigidBodyDynamics.Joint… => [0.3023354616732832]

In [7]:
num_positions(x)

8

In [8]:
num_velocities(x)

7

In [9]:
cache = MechanismStateCache(mechanism, x);

RigidBodyDynamics.CartesianFrame3D("world")
RigidBodyDynamics.CartesianFrame3D("after_1")


In [10]:
relative_transform(cache, joint2.frameAfter, body1.frame)

RigidBodyDynamics.Transform3D{Float64}(RigidBodyDynamics.CartesianFrame3D("after_2"),RigidBodyDynamics.CartesianFrame3D("body1"),0.6091484295850903 + 0.3147851549100663im + 0.5624194320047656jm + 0.4620961799081452km,Vec(0.22682906360745578,0.7831275047519294,0.7150371368169544))

In [11]:
center_of_mass(mechanism, cache)

RigidBodyDynamics.Point3D{Float64}(RigidBodyDynamics.CartesianFrame3D("world"),Vec(-1.0538679151814974,0.4804818683587106,1.2330514922488731))

In [12]:
mass(mechanism)

1.445267503937063

In [13]:
transform(cache, Point3D{Float64}(joint2.frameAfter), root(mechanism).frame)

RigidBodyDynamics.Point3D{Float64}(RigidBodyDynamics.CartesianFrame3D("world"),Vec(-1.1244598118634213,0.6765210945373247,0.17174448679765747))

In [14]:
relative_twist(cache, body2, body1)

RigidBodyDynamics.Twist{Float64}(RigidBodyDynamics.CartesianFrame3D("after_2"),RigidBodyDynamics.CartesianFrame3D("after_1"),RigidBodyDynamics.CartesianFrame3D("world"),Vec(-0.1298502767353893,-0.004776670345753353,-0.032966006614218585),Vec(-0.02148183208004284,-0.059370018733765795,0.09321762518131949))

In [15]:
relative_twist(cache, body1, world) + relative_twist(cache, body2, body1) + relative_twist(cache, world, body2)

RigidBodyDynamics.Twist{Float64}(RigidBodyDynamics.CartesianFrame3D("world"),RigidBodyDynamics.CartesianFrame3D("world"),RigidBodyDynamics.CartesianFrame3D("world"),Vec(0.0,0.0,0.0),Vec(0.0,0.0,0.0))

In [16]:
geometric_jacobian(cache, path(mechanism, body2, world))

RigidBodyDynamics.GeometricJacobian{Float64}(RigidBodyDynamics.CartesianFrame3D("world"),RigidBodyDynamics.CartesianFrame3D("after_2"),RigidBodyDynamics.CartesianFrame3D("world"),6x7 Array{Float64,2}:
  0.429491    0.913297   -0.402005  …  -0.0       -0.0       -0.0      
  0.0157992  -0.347465   -0.852827     -0.0       -0.0       -0.0      
  0.109038   -0.212499   -0.333284     -0.0       -0.0       -0.0      
  0.071053    0.0138353   0.135899      0.913297  -0.402005   0.0654207
  0.196371    0.295048   -0.21771      -0.347465  -0.852827  -0.389813 
 -0.308325   -0.42298     0.393169  …  -0.212499  -0.333284   0.918567 )

In [17]:
mass_matrix(cache)

7x7 Array{Float64,2}:
 -0.91335     0.210748     -0.00326905  …   0.0479158    -0.252227 
 -0.0895638   1.4463        0.985291       -0.59604      -0.583173 
  0.119675    0.769494      0.340541       -6.93889e-18   0.337634 
 -6.245e-17  -0.475769     -0.0479158       0.0           0.301588 
  0.475769   -5.55112e-17   0.59604        -5.55112e-17   0.571274 
  0.0479158  -0.59604       0.0         …   0.817964      0.142814 
 -0.252227   -0.583173      0.337634        0.142814      0.0797192

In [20]:
[J1 J2]

RigidBodyDynamics.GeometricJacobian{Float64}(RigidBodyDynamics.CartesianFrame3D("after_2"),RigidBodyDynamics.CartesianFrame3D("world"),RigidBodyDynamics.CartesianFrame3D("world"),6x7 Array{Float64,2}:
 -0.913297    0.402005  -0.0654207  …  0.0        0.0        -0.429491 
  0.347465    0.852827   0.389813      0.0        0.0        -0.0157992
  0.212499    0.333284  -0.918567      0.0        0.0        -0.109038 
 -0.0138353  -0.135899  -0.641939      0.402005  -0.0654207  -0.071053 
 -0.295048    0.21771   -0.213307      0.852827   0.389813   -0.196371 
  0.42298    -0.393169  -0.0448022  …  0.333284  -0.918567    0.308325 )

In [21]:
geometric_jacobian(cache, path(mechanism, world, body1))

RigidBodyDynamics.GeometricJacobian{Float64}(RigidBodyDynamics.CartesianFrame3D("after_1"),RigidBodyDynamics.CartesianFrame3D("world"),RigidBodyDynamics.CartesianFrame3D("world"),6x6 Array{Float64,2}:
 -0.913297    0.402005  -0.0654207   0.0       0.0        0.0      
  0.347465    0.852827   0.389813    0.0       0.0        0.0      
  0.212499    0.333284  -0.918567    0.0       0.0        0.0      
 -0.0138353  -0.135899  -0.641939   -0.913297  0.402005  -0.0654207
 -0.295048    0.21771   -0.213307    0.347465  0.852827   0.389813 
  0.42298    -0.393169  -0.0448022   0.212499  0.333284  -0.918567 )

In [22]:
geometric_jacobian(cache, path(mechanism, body1, body2))

RigidBodyDynamics.GeometricJacobian{Float64}(RigidBodyDynamics.CartesianFrame3D("after_2"),RigidBodyDynamics.CartesianFrame3D("after_1"),RigidBodyDynamics.CartesianFrame3D("world"),6x1 Array{Float64,2}:
 -0.429491 
 -0.0157992
 -0.109038 
 -0.071053 
 -0.196371 
  0.308325 )

In [23]:
motion_subspace(cache, joint2)

RigidBodyDynamics.GeometricJacobian{Float64}(RigidBodyDynamics.CartesianFrame3D("after_2"),RigidBodyDynamics.CartesianFrame3D("after_1"),RigidBodyDynamics.CartesianFrame3D("world"),6x1 Array{Float64,2}:
 -0.429491 
 -0.0157992
 -0.109038 
 -0.071053 
 -0.196371 
  0.308325 )

In [24]:
J = geometric_jacobian(cache, path(mechanism, world, body2))

RigidBodyDynamics.GeometricJacobian{Float64}(RigidBodyDynamics.CartesianFrame3D("after_2"),RigidBodyDynamics.CartesianFrame3D("world"),RigidBodyDynamics.CartesianFrame3D("world"),6x7 Array{Float64,2}:
 -0.913297    0.402005  -0.0654207  …  0.0        0.0        -0.429491 
  0.347465    0.852827   0.389813      0.0        0.0        -0.0157992
  0.212499    0.333284  -0.918567      0.0        0.0        -0.109038 
 -0.0138353  -0.135899  -0.641939      0.402005  -0.0654207  -0.071053 
 -0.295048    0.21771   -0.213307      0.852827   0.389813   -0.196371 
  0.42298    -0.393169  -0.0448022  …  0.333284  -0.918567    0.308325 )

In [25]:
J...

LoadError: LoadError: unsupported or misplaced expression ...
while loading In[25], in expression starting on line 1

In [26]:
hcat(J...)

LoadError: LoadError: MethodError: `start` has no method matching start(::RigidBodyDynamics.GeometricJacobian{Float64})
while loading In[26], in expression starting on line 1